[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/racousin/rag_attack/blob/main/partie1_rag.ipynb)

# RAG Agentique avec LangGraph - VéloCorp

Ce notebook démontre un système RAG simple permettant d'obtenir des informations sur la base des documentations techniques de VéloCorp.
Il contient les sections suivantes:
- Installation
- Implémentation d'un RAG local
    1) chargement des données 
    2) chunking
    3) vectorisation des chunks
    4) génération
- Implémentation d'un RAG sur Azure (optionel)

# Installation

In [1]:
# Installations nécessaires
!pip install azure-core \
azure-identity \
azure-search\
azure-search-documents \
certifi\
python-dotenv \
faiss-cpu \
ipykernel \
langchain-community \
langchain-core \
langchain-huggingface \
langchain-openai \
langgraph \
pyodbc\
requests \
sentence-transformers


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# Nécessaire d'effecture ceci avant d'importer Azure Search sinon Azure utilise
# sa nomenclature par défaut
import os
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "embedding"

In [3]:
# Imports standards :
import json
import logging
import os
import re
from typing import Dict, Optional

# Imports Azure
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

try:
    from azure.search.documents.models import VectorizedQuery
except ImportError:
    VectorizedQuery = None

# Imports third-party
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import AzureChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Optional, Dict, Any
try: 
    from google.colab import files
except:
    pass 

#  Impléméntation d'un RAG local

### Etape 1 : Chargement des données

In [4]:
# Récupérer le fichier local
uploaded = files.upload()

# Récupérer le nom du fichier uploadé dans colab
filename = next(iter(uploaded))

NameError: name 'files' is not defined

In [ ]:
def manuals_json_loader(json_path: str) -> List[Dict]:
    """
    Charge uniquement les documents de type 'manuel_technique' à partir d'un fichier JSON unifié.

    Args:
        json_path (str): Chemin vers le fichier JSON fusionné contenant tous les documents.

    Returns:
        list of dict: Liste de dictionnaires contenant l'id, les métadonnées et le texte des manuels techniques.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    manuals = []  

    for key, item in data.items():
        meta = item.get("metadata", {})
        # On vérifie que le document est bien un manuel technique
        if meta and meta.get("type", "").lower() == "manuel_technique":
            manuals.append({
                "id": key,                
                "metadata": meta,         
                "text": item.get("text", "") 
            })
    return manuals

manuals = manuals_json_loader(filename)
manuals

[{'id': 'manuel_urbain_classic',
  'metadata': {'type': 'manuel_technique',
   'model': 'Urbain-Classic',
   'product_line': 'Urbain',
   'version': '2.5',
   'last_updated': '2025-03-13',
   'word_count': 368},
  'text': "Document ID: 20\nCreated: 2025-03-13\nFilename: manuel_urbain_classic.txt\n--------------------------------------------------\n\nManuel Technique - Urbain-Classic\n        \nBienvenue dans le manuel technique de votre Urbain-Classic. Ce document contient toutes les informations nécessaires pour l'utilisation, l'entretien et la maintenance de votre vélo.\n\nSPÉCIFICATIONS TECHNIQUES - Urbain-Classic\n\nPoids: 17.2 kg\nMatériau du cadre: acier\nTailles disponibles: S, M, L, XL\nCouleurs: noir, blanc, bleu\nGarantie: 12 mois\n\nComposants:\n- Freins: Freins à disque hydrauliques\n- Transmission: Dérailleur 21 vitesses\n- Roues: Jantes aluminium double paroi\n- Pneus: Pneus tout-terrain 700x35c\n- Selle: Selle ergonomique avec gel\n\nMONTAGE ET INSTALLATION\n\n1. Déballa

### Etape 2: chunking

Nous allons procéder au chunking (découpage) des documents, la fonction suivante implante un recursive chunking, mais il existe d'autres méthodes (comme vu précédemment). Le récursive chunking est une bonne base qui donne des résultats suffisants dans la majorité des cas.  
Ici, le texte va d'abord être nettoyé des caractères spéciaux, puis découper avec un récursive chunker. Il permet de créer des chunks de longueur fixe avec un certain nombre des premiers mots en recouvrant pour ne pas avoir de coupure trop brutale (ex: éviter de couper une phrase en 2)

In [6]:
def chunking(manuals: List[Dict[str, Any]], chunk_size: int = 1000, chunk_overlap: int = 200) -> List[Dict[str, Any]]:
    """
    Découpe une liste de manuels en chunks textuels avec recouvrement et
    enrichit chaque chunk du contexte produit (modèle, type, ligne de produit).

    Args:
        manuals: liste de dicts représentant les manuels. Chaque dict doit
                contenir au minimum les clés "id", "metadata" et "text".
        chunk_size: taille maximale (en caractères) d'un chunk.
        chunk_overlap: nombre de caractères de recouvrement entre chunks adjacents.

    Returns:
        Liste de dictionnaires avec les clés: doc_id, chunk_id, metadata, text
    """
    if not manuals:
        return []

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " "]
    )

    all_chunks: List[Dict[str, Any]] = []
    for doc in manuals:
        text = doc.get("text", "")
        if not text.strip():
            continue

        # Nettoyage des séparateurs visuels (lignes de tirets ou espaces répétés)
        clean_text = re.sub(r"(^[-\s]{3,}$\n?)+", "", text, flags=re.MULTILINE)

        # Découpage en chunks avec recouvrement
        chunks = splitter.split_text(clean_text)

        for i, chunk in enumerate(chunks):
            # Créer le header de contexte séparement
            header = f"Modèle : {doc['metadata'].get('model', '')}\n"
            header += f"Type : {doc['metadata'].get('type', '')}\n"
            header += f"Ligne de produit : {doc['metadata'].get('product_line', '')}\n"
            # Combiner le header avec UNIQUEMENT le chunk (pas le texte entier)
            chunk_text = header + chunk
            
            # Copie des métadonnées pour éviter mutations involontaires
            metadata = dict(doc.get("metadata", {}) or {})
            # Ajout d'un lien explicite vers le document source dans les métadatas
            metadata.setdefault("doc_id", doc.get("id"))

            all_chunks.append({
                "doc_id": doc.get("id"),
                "chunk_id": f"{doc.get('id')}_chunk_{i}",
                "metadata": metadata,
                "text": chunk_text
            })

    return all_chunks

Nous allons maintenant procéder au chunking des documents, n'hésitez pas à jouer sur les paramètres chunk_size et chunk_overlap pour voir l'influence sur les chunks.  
Ces paramètres sont très importants car ils permettront d'affiner la réponse finale générée par le RAG:  
- des chunks courts permettent d'obtenir des résultats pertinents sur un sujet précis mais peuvent parfois manquer de contexte global  
- des chunks longs permettent d'avoir un contexte complet mais peuvent engendrer des hallucinations en créant un prompt de génération trop long  

Nous déterminons ces paramètres de manière empirique dans la plupart des cas, mais pouvons aussi les affiner automatiquement lorque l'on dispose d'un dataset d'évaluation (type FAQ) et d'une fonction d'évaluation/scoring


In [8]:
all_chunks = chunking(
    manuals,
    chunk_size=1000,    #Paramètre à tester
    chunk_overlap=200   #Paramètre à tester
    )

# Affichage d'exemples
for chunk in all_chunks[:10]:
    print("="*100)
    print(chunk["text"])

Modèle : Urbain-Classic
Type : manuel_technique
Ligne de produit : Urbain
Document ID: 20
Created: 2025-03-13
Filename: manuel_urbain_classic.txt
Manuel Technique - Urbain-Classic
Bienvenue dans le manuel technique de votre Urbain-Classic. Ce document contient toutes les informations nécessaires pour l'utilisation, l'entretien et la maintenance de votre vélo.

SPÉCIFICATIONS TECHNIQUES - Urbain-Classic

Poids: 17.2 kg
Matériau du cadre: acier
Tailles disponibles: S, M, L, XL
Couleurs: noir, blanc, bleu
Garantie: 12 mois

Composants:
- Freins: Freins à disque hydrauliques
- Transmission: Dérailleur 21 vitesses
- Roues: Jantes aluminium double paroi
- Pneus: Pneus tout-terrain 700x35c
- Selle: Selle ergonomique avec gel

MONTAGE ET INSTALLATION

1. Déballage
   - Vérifiez que tous les composants sont présents
   - Inspectez le cadre pour détecter d'éventuels dommages
   - Rassemblez les outils nécessaires

2. Installation de la roue avant
   - Insérez la roue dans la fourche
   - Serrez 

### Etape 3: Vectorisation des chunks
Désormais, nous allons vectoriser (embedding) nos chunks et les stocker pour créer notre base de connaissance. Ces vecteurs sont stockés dans une base appellée Vector Store qui est un type de base de données optimisée pour ce type de données. Elle dispose d'algorithme permettant de retrouver efficacement les passages les plus pertinents par similarité de sens, même si les mots exacts ne sont pas présents dans la requête de l’utilisateur (retrieval).

Dans ce TP, nous utilison FAISS, qui est une bibliothèque open source permettant la recherche ultra-rapide de similarité entre vecteurs dans de grands volumes de données.
On l’utilise pour retrouver efficacement les passages ou documents les plus pertinents à partir d’une requête, même dans des bases contenant des millions de textes ou d’images.
Parmi les alternatives, on trouve ChromaDB, mongoDB, PGvector, redis, , , ... 


In [9]:
def create_faiss_vectorstore(
        embedded_chunks: List[Dict[str, Any]],
        model_name: str = 'sentence-transformers/all-MiniLM-L6-v2'
    ) -> FAISS:
    """
    Initialise le vector store, vectorise les chunks et les stocke.

    Args:
        embedded_chunks: la liste des chunks créés précédement
        model_name: nom du modèle à utiliser pour l'embedding
    Returns:
        Le vector store faiss
    """
    # 1. Création des objets Document avec contenu et métadonnées
    docs = [
        Document(
            page_content=chunk["text"],
            metadata=chunk["metadata"]
        )
        for chunk in embedded_chunks
    ]

    # 2. Instanciation du modèle d'embedding et vectorisation des chunks
    embedding_fn = HuggingFaceEmbeddings(model_name=model_name)


    # 3. Stockage des  du vector store FAISS
    vectorstore = FAISS.from_documents(docs, embedding_fn)

    return vectorstore

vectorstore = create_faiss_vectorstore(all_chunks)
print(f"Vector store FAISS créé avec {vectorstore.index.ntotal} chunks.")
print(f"Dimension des embeddings : {vectorstore.index.d}")
print(f"Exemple d'embedding : {vectorstore.index.reconstruct(0)}")

/var/folders/vg/mwr3q69x59q89ybp7kdv2b700000gp/T/ipykernel_73749/2838648013.py:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_fn = HuggingFaceEmbeddings(model_name=model_name)


Vector store FAISS créé avec 40 chunks.
Dimension des embeddings : 384
Exemple d'embedding : [-6.39532134e-02  6.88244104e-02  2.16990262e-02 -3.82395387e-02
 -5.62896840e-02 -3.46306935e-02 -4.01799455e-02  1.50626898e-01
 -3.75493877e-02 -1.86245833e-02  1.94938350e-02  1.42349245e-03
  3.10550500e-02 -1.30971139e-02 -4.66126725e-02  1.97264683e-02
  2.95842476e-02 -1.12555316e-02  1.31323552e-02  5.52595854e-02
  1.17211983e-01 -3.27336378e-02 -2.63883788e-02 -1.29726436e-03
 -8.50468427e-02  1.44928554e-02 -9.19697620e-03  8.36963803e-02
 -2.57104617e-02 -1.73001274e-01  7.56680453e-03  5.93945198e-02
 -4.89427149e-02 -6.48005903e-02  2.21751407e-02 -4.77569513e-02
 -7.51574412e-02 -5.09632863e-02 -8.20729733e-02  7.22378120e-02
  7.61495763e-03 -5.18737733e-02 -3.51051539e-02  7.81464484e-03
 -6.83395332e-03  2.66488753e-02 -3.27349976e-02  7.53232986e-02
 -2.40764897e-02  4.43060249e-02 -2.06668843e-02 -1.06018754e-02
  6.07157797e-02  5.69669809e-03 -4.19344846e-03 -7.11843232e-

Les 40 chunks ont été vectorisés avec un modèle all-MiniLM-L6-v2 disponible sur Hugging Face (plateforme open source hebergeant une grande quantité de modèle IA utilisable librement). Il existe une grande quantité de modèles permettant de vectoriser des chunks: E5, text-embedding (openAI), BERT, ...  
Dans notre cas, le modèle utilisé créé des vecteurs de dimension 384 pour representer nos chunks, mais attention un autre modèle d'embedding pourrait avoir une autre dimension.

In [10]:
# Vous pouvez essayer ici de manipuler le modèle d'embedding et voir l'évolution des vecteurs en fonction de vos inputs
example_text = ["Phrase de test"]
vectorstore.embeddings.embed_documents(example_text)[0]


[-0.019201353192329407,
 0.05179671570658684,
 0.05539053678512573,
 0.011876818723976612,
 0.020779689773917198,
 0.0007148848380893469,
 0.1300562471151352,
 0.009340181015431881,
 0.03354818746447563,
 0.01540105976164341,
 0.13285452127456665,
 -0.08587942272424698,
 0.02730434015393257,
 -0.014276807196438313,
 0.015483113005757332,
 -0.07269447296857834,
 0.06184579059481621,
 0.0011288989335298538,
 -0.007486025802791119,
 0.02465115301311016,
 0.055983223021030426,
 0.00905511062592268,
 -0.09904667735099792,
 0.04716170206665993,
 0.045458607375621796,
 0.04703236371278763,
 -0.06344479322433472,
 0.058493733406066895,
 0.033416543155908585,
 0.00010275186650687829,
 0.030536916106939316,
 0.06458554416894913,
 0.019313573837280273,
 0.08643624931573868,
 0.06922352313995361,
 0.018661394715309143,
 -0.01576383411884308,
 -0.002477696631103754,
 0.01632366143167019,
 0.04557834938168526,
 -0.05140039324760437,
 -0.07692334055900574,
 0.0189953800290823,
 -0.0038937795907258987

Nous allons maintenant persister cet index en le stockant localement, le but est d'ici de simuler un serveur de base de donnée afin de pouvoir recharger la base si besoin.

In [ ]:
# Chemin de sauvegarde 
persist_directory = "/content/data/faiss_vectorstore"
os.makedirs(persist_directory, exist_ok=True)
print(f"Dossier créé à : {persist_directory}")

# Sauvegarde du vector store
vectorstore.save_local(persist_directory)
print(f"Vector store FAISS sauvegardé à : {persist_directory}")

Dossier créé à : faiss_vectorstore
Vector store FAISS sauvegardé à : faiss_vectorstore


### Etape 4: Retrieval
Maintenant que nous avons préparé nos données, il est est temps de les utiliser pour générer des réponses contextualisées à notre domaine.  
Le principe du retrieval est de retrouver les chunks dans notre base de connaissance les plus similaires par rapport à notre question (query).  Nous commencons par vectoriser la question avec le même modèle d'embedding utilisé précédemment puis nous comparons le vecteur de la question avec nos embeddings et retournons les k résultats les plus similaires (comparaison vectorielle: cosine similarity, distance vectorielle, ...).  
Dans notre cas, la base de donnée faiss nous permet de simplifier le retrieval en vectorisant la question et en faisant la recherche vectorielle avec une seule fonction similarity search

Le paramètre k permet de déterminer le nombre de documents les plus similaires à extraire de la base, ce paramètre se détermine aussi de manière empirique, mais peut se déterminer automatiquement de la même manière que pour les paramètres du chunking.


In [12]:
query = "Quel est le poids du vélo Sport-Elite ?" #  N'hésitez pas à tester d'autres questions ici 

# Retrieval des chunks les plus similaires (vectorisation de la query + recherche vectorielle)
results = vectorstore.similarity_search(
    query, 
    k=2 # Paramètre à expérimenter
    )

for i, doc in enumerate(results):
    print(f"\n--- Résultat #{i+1} ---")
    print("Texte du chunk :")
    print(doc.page_content)


--- Résultat #1 ---
Texte du chunk :
Modèle : Sport-Elite
Type : manuel_technique
Ligne de produit : Sport
Problème: Roue qui frotte
Solution: Vérifiez l'alignement de la roue et des freins

Problème: Bruit anormal
Solution: Identifiez la source, vérifiez le serrage des composants

Pour tout problème persistant, contactez notre service technique au 01 23 45 67 89.

--- Résultat #2 ---
Texte du chunk :
Modèle : Sport-Elite
Type : manuel_technique
Ligne de produit : Sport
Document ID: 5
Created: 2024-08-28
Filename: manuel_sport_elite.txt
Manuel Technique - Sport-Elite
Bienvenue dans le manuel technique de votre Sport-Elite. Ce document contient toutes les informations nécessaires pour l'utilisation, l'entretien et la maintenance de votre vélo.

SPÉCIFICATIONS TECHNIQUES - Sport-Elite

Poids: 8.5 kg
Matériau du cadre: carbone
Tailles disponibles: S, M, L, XL
Couleurs: gris, bleu
Garantie: 24 mois

Composants:
- Freins: Freins à disque hydrauliques
- Transmission: Dérailleur 21 vitesses


### Etape 4: Génération
Maintenant que nous avons extrait les documents les plus pertinents par rapport à notre question, il est temps de formuler une réponse contextualisée en les utilisant.  
Nous allons instancier notre endpoint LLM qui permet de générer du texte, ici nous utiliserons un endpoint openAI gpt4o. Ce modèle apporte de bonnes performances sur des tâches de génération de texte simple comme le RAG.  

D'autres modèles peuvent être utilisés et dépendent principalement du fournisseur cloud disponible: gemini (GCP), claude (AWS), gpt (Azure). Il est aussi possible d'utiliser aussi des modèles open source hébergés on premise, mais ceux-ci nécessitent une infrastructure conséquente et couteuse pour les utiliser (GPU), on note notamment les modèles mistral, llama, gwen, ...

In [ ]:
# Demander la confiuguration 
config = {

}

#Instanciation du LLM
llm = AzureChatOpenAI(
    azure_endpoint  = config["openai_endpoint"],
    api_key         = config["openai_key"],
    deployment_name = config["chat_deployment"],
    api_version     = "2024-02-15-preview"
)

In [14]:
text_input = "Bonjour, comment ça va ?"
print(llm.invoke(text_input).content)

Bonjour ! Je vais bien, merci. Comment puis-je vous aider aujourd'hui ?


Nous allons maintenant construire le prompt de retrieval, un bon prompt doit toujours contenir à minima les informations suivantes:
- la persona du répondant et ses compétences: ici un expert technique vélo
- une tâche: répondre à une question posée (query)
- le contexte: les sources à utiliser pour réaliser sa tâche (chunks)
- instructions: règles à respecter pour générer sa réponse (format, inputs, contraintes, citations, ...)

In [ ]:
#A vous de jouer !
prompt ="""
Ecrivez ici votre prompt de retrieval en vous basant sur les indications précédentes.
Les deux variables suivantes (entre accolades) doivent être utilisées sinon le code ne fonctionnera pas :
{chunks}
{query}
"""

Testons maintenant l'ensemble de votre chaine de RAG avec une question

In [16]:
query = "Quel est le poids vélo Urbain-Classic ?"
results = vectorstore.similarity_search(query, k=2) 
chunks = "\n\n".join([doc.page_content for doc in results])

response = llm.invoke(prompt.format(chunks=chunks, query=query))
print(response.content)
#Ici, la réponse attendue est 17.2 kg, si ce n'est pas le cas, essayez d'ajuster votre prompt

Le poids du vélo Urbain-Classic est de 17.2 kg. (Source: SPÉCIFICATIONS TECHNIQUES - Urbain-Classic)


A titre comparatif, voici le résultat fourni par un LLM à la même question mais sans l'ajout du contexte

In [17]:
print(llm.invoke(query).content)

Je ne dispose pas d'informations spécifiques sur le modèle "Urbain-Classic". Le poids des vélos urbains peut varier en fonction de la marque, du modèle et des matériaux utilisés. En général, les vélos urbains pèsent entre 12 et 18 kilogrammes. Pour obtenir le poids exact du modèle "Urbain-Classic", je vous recommande de consulter les spécifications techniques fournies par le fabricant ou le distributeur.


Finalement, testons les limites de notre RAG dans le cas où l'information n'est pas présente dans notre base de connaissance.  
Le comportement souhaité est que le LLM ne réponde pas, car il ne dispose pas de l'information et ne dois surtout pas inventer du contenu comme dans l'exemple précédent.

In [18]:
query = "De quel matériau est fabriqué le vélo urbain-confort ?"
results = vectorstore.similarity_search(query, k=2)
chunks = "\n\n".join([doc.page_content for doc in results])
response = llm.invoke(prompt.format(chunks=chunks, query=query))
print(response.content)
#Ici, le modèle doit répondre qu'il ne connait pas l'information, si ce n'est pas le cas, essayez d'ajuster votre prompt

L’information ne figure pas dans la documentation fournie.


#### Correction
Voici un exemple de prompt de retrieval que vous pouvez utiliser :

In [15]:
prompt = """
Tu es un assistant expert, spécialisé dans la documentation technique des vélos.

Réponds de façon précise et concise à la question utilisateur suivante,
en t’appuyant STRICTEMENT et EXCLUSIVEMENT sur les extraits ci-dessous (issus de la documentation technique officielle) :
\"\"\"
{chunks}
\"\"\"

INSTRUCTIONS :
- Ne donne aucune information qui ne provient pas explicitement de ces extraits.
- Si la réponse exacte (par exemple, une valeur numérique, un modèle, un tableau) est présente, cite-la textuellement et indique d'où elle provient.
- Si plusieurs passages sont nécessaires pour une réponse complète, combine-les de façon claire.
- Si l’information demandée n’apparaît pas dans les extraits, dis simplement : « L’information ne figure pas dans la documentation fournie. »
- Reste factuel, sans extrapolation ni interprétation.

Question :
{query}
"""

## Implémentation d'un RAG sur Azure 
#### Ce RAG suit la même logique que précédemment, mais en utilisant Azure. Il est encapsulé dans une classe afin de pouvoir le réutiliser comme Tool pour notre RAG agentique.

#### Instanciation des LLMs via Azure

In [ ]:
llm = AzureChatOpenAI(
    azure_endpoint  = config["openai_endpoint"],
    api_key         = config["openai_key"],
    deployment_name = config["chat_deployment"],
    api_version     = "2024-02-15-preview"
)


#### Création d'une classe pour récupérer uniquement les manuels utilisateurs

In [ ]:
# -----------------------------------------------------------------------------
# AzureSearchFilteredRetriever
# -----------------------------------------------------------------------------
# Petit utilitaire pour interroger directement un index Azure Cognitive Search
# en mode "vector search" + filtre OData (ex: ne récupérer que les documents
# dont le champ 'type' vaut 'manuel').
#
# Pourquoi ce helper ?
# - Le wrapper LangChain AzureSearch ne propage pas toujours bien les filtres.
# - Ici on utilise le SDK officiel azure-search-documents pour garder le contrôle.
#
# Paramètres attendus :
#   endpoint   : URL du service Azure Search (ex: "https://xxx.search.windows.net")
#   index_name : nom de l'index (ex: "documents")
#   key        : clé admin ou query key autorisée
#   embed_fn   : fonction Python qui transforme un texte en vecteur (liste de floats)
#
# Méthode principale :
#   search_manuals(query, k=5, filter_expr="type eq 'manuel'")
#     -> renvoie les résultats bruts (SearchResult-like) retournés par Azure.
# -----------------------------------------------------------------------------

class AzureSearchFilteredRetriever:
    def __init__(self, endpoint: str, index_name: str, key: str, embed_fn):
        # Enregistre les paramètres de connexion / embedding
        self.endpoint = endpoint
        self.index_name = index_name
        self.embed_fn = embed_fn
        # Client natif Azure Search
        self.client = SearchClient(endpoint, index_name, AzureKeyCredential(key))

    def search_manuals(self, query: str, k: int = 5, filter_expr: str = "type eq 'manuel'"):
        # 1. embed la requête
        vec = self.embed_fn(query)
        # 2. construit l’appel selon SDK
        if VectorizedQuery is not None:
            # Nouveau SDK
            vq = VectorizedQuery(vector=vec, k=k, fields="embedding")
            results = self.client.search(
                search_text="*",  # pas de fulltext, vector only
                vector_queries=[vq],
                filter=filter_expr,
            )
        else:
            # Ancien SDK
            from azure.search.documents.models import Vector  # fallback
            v = Vector(value=vec, k=k, fields="embedding")
            results = self.client.search(
                search_text="*",
                vector=v,
                filter=filter_expr,
            )

        docs = []
        for r in results:
            # r est un dict-like
            docs.append(r)
        return docs


#### Création d'une classe pour le RAG

In [ ]:
# -----------------------------------------------------------------------------
# Configuration logging simple pour afficher les étapes du pipeline.
# -----------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)


# -----------------------------------------------------------------------------
# RAGPipelineAzure
# -----------------------------------------------------------------------------
# Ce pipeline combine :
#   - un modèle d'embedding HuggingFace (utilisé pour vectoriser les requêtes)
#   - un LLM hébergé sur Azure OpenAI (chat completions)
#   - un index Azure Cognitive Search contenant des documents vectorisés
#
# self.search_client : client natif Azure + retrieve_filtered() -> filtre OData
#
# Usage typique :
#   pipeline = RAGPipelineAzure(...config Terraform...)
#   réponse = pipeline.answer("Ma question", k=5)
#
# NB : l'index contient un champ 'type' (filterable=True) et
#       un champ vectoriel 'embedding' (dimensions cohérentes avec l'embedding_fn).
# -----------------------------------------------------------------------------

class RAGPipelineAzure:
    def __init__(
        self,
        model_name: str,
        api_key: str,
        api_base: str,
        api_version: str,
        deployment_name: str,
        config: Dict[str, Any],
        index_name: str = "documents",
    ) -> None:

        """
        Initialise la pipeline RAG avec Azure Cognitive Search vectoriel.
        """
        # Sauvegarde du nom du modèle d'embedding HuggingFace
        self.model_name = model_name
        # Initialise la fonction d'embedding (requêtes & docs)
        self.embedding_fn = HuggingFaceEmbeddings(model_name=model_name)
        # Client LLM Azure OpenAI (déploiement chat)
        self.llm = AzureChatOpenAI(
            azure_endpoint=api_base,
            api_key=api_key,
            api_version=api_version,
            deployment_name=deployment_name,
        )
        # Nom de l'index Search
        self.index_name = index_name

        # Client natif Search pour filtrage propre
        self.search_client = SearchClient(
            config["search_endpoint"],
            index_name,
            AzureKeyCredential(config["search_key"])
        )

        # Petite lambda utilitaire : embed une requête texte -> vecteur (liste de floats)
        self._embed_query = lambda txt: self.embedding_fn.embed_query(txt)

    def retrieve_filtered(
        self,
        query: str,
        k: int = 5,
        doc_type: Optional[str] = None
    ) -> List[Document]:

        """Vector search côté Azure + filtre OData sur type='...'."""
        # Embed de la requête utilisateur
        vec = self._embed_query(query)
        # Expression de filtre OData (doit matcher un champ filterable dans l'index)
        filter_expr = f"type eq '{doc_type}'" if doc_type else None

        results_iter = None
        try:
            # Tentative nouvelle API
            from azure.search.documents.models import VectorizedQuery
            vq = VectorizedQuery(vector=vec, k=k, fields="embedding")
            results_iter = self.search_client.search(
                search_text="*",  # vector-only pattern
                vector_queries=[vq],
                filter=filter_expr,
            )
        except ImportError:
            # Ancienne API
            from azure.search.documents.models import Vector
            v = Vector(value=vec, k=k, fields="embedding")
            results_iter = self.search_client.search(
                search_text="*",
                vector=v,
                filter=filter_expr,
            )
        except Exception as e:
            # Toute autre erreur (réseau, auth, schéma, etc.)
            logger.error(f"Recherche Azure Search échouée: {e}")
            raise

        # Conversion vers objets LangChain Document pour réutiliser la suite du pipeline
        from langchain.schema import Document
        docs = []
        for r in results_iter:
            # r est SearchResult -> accès dict-like
            docs.append(
                Document(
                    page_content=r.get("content", "") if hasattr(r, "get") else r["content"],
                    metadata={
                        "id": r["id"] if "id" in r else getattr(r, "id", None),
                        "filename": r.get("filename") if hasattr(r, "get") else r.get("filename", None) if isinstance(r, dict) else None,
                        "type": r.get("type") if hasattr(r, "get") else r.get("type", None) if isinstance(r, dict) else None,
                        "created": r.get("created") if hasattr(r, "get") else r.get("created", None) if isinstance(r, dict) else None,
                        "document_id": r.get("document_id") if hasattr(r, "get") else r.get("document_id", None) if isinstance(r, dict) else None,
                    },
                )
            )
        # On ne retourne que les k premiers (Azure en a déjà retourné k, mais on sécurise)
        return docs[:k]

    def answer(self, query: str, k: int = 5) -> str:
        """Renvoie une réponse générée par le LLM sur la base des docs filtrés."""
        try:
            # Récupération des meilleurs chunks filtrés côté Azure (type='manuel')
            results = self.retrieve_filtered(query, k=k, doc_type="manuel")
            # Concaténation du contenu des chunks pour former le contexte
            context = "\n\n".join([doc.page_content for doc in results])
            # Prompt instructif (contexte + question)
            prompt = f"""
                Tu es un assistant expert, spécialisé dans la documentation technique des vélos.

                Réponds de façon précise et concise à la question utilisateur suivante,
                en t’appuyant STRICTEMENT et EXCLUSIVEMENT sur les extraits ci-dessous (issus de la documentation technique officielle) :
                \"\"\"
                {context}
                \"\"\"

                INSTRUCTIONS :
                - Ne donne aucune information qui ne provient pas explicitement de ces extraits.
                - Si la réponse exacte (par exemple, une valeur numérique, un modèle, un tableau) est présente, cite-la textuellement et indique d'où elle provient.
                - Si plusieurs passages sont nécessaires pour une réponse complète, combine-les de façon claire.
                - Si l’information demandée n’apparaît pas dans les extraits, dis simplement : « L’information ne figure pas dans la documentation fournie. »
                - Reste factuel, sans extrapolation ni interprétation.

                Question :
                {query}
                """
            # Appel LLM Azure OpenAI
            logger.info("Envoi du prompt au LLM Azure OpenAI...")
            response = self.llm.invoke(prompt)
            logger.info("Retour LLM récupéré.")
            content = getattr(response, "content", None)
            if not content:
                logger.warning("Pas de contenu retourné par le LLM ou réponse vide.")
                return "Le LLM n'a rien répondu (réponse vide, sans contenu)"
            return content
        except Exception as e:
            logger.error(f"Erreur lors de la génération de la réponse LLM : {e}")
            return "Une erreur s'est produite lors de la génération de la réponse."


In [ ]:
pipeline = RAGPipelineAzure(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # Le modèle d'embedding HuggingFace utilisé à l'indexation
    api_key=config["openai_key"],                         # Clé API Azure OpenAI sortie du terraform
    api_base=config["openai_endpoint"],                   # Endpoint Azure OpenAI sorti du terraform
    api_version="2024-02-15-preview",                     # Version API Azure OpenAI (à ajouter dans ta classe, cf plus bas)
    deployment_name=config["chat_deployment"],            # Nom du déploiement Azure OpenAI
    config=config,
    index_name="documents"
)

### Tests simple pour vérifier le résultat

In [ ]:
# Réponse
print(pipeline.answer("En quelles couleurs est disponible le modèle urbain classic ?"))

In [ ]:
print(pipeline.answer("Quel est le poids du Urbain-Confort ?"))

In [ ]:
print(pipeline.answer("Que faire si la chaîne du vélo Pro-Livraison saute ?"))

### Test pour une information absente des manuels

In [ ]:
print(pipeline.answer("Quel est le diamètre des roues du vélo Pro-Livraison ?"))